задача - предсказание популярности по жанру, году etc

План:
1. Открыть файл song_info.json, распасрсить в пандас
2. Нужны: дата, просмотры, репосты, теги
3. Делим выборку на трейн и тест
4. Создаем модель (линейная регрессия), передаем ей параметры 
5. Пробуем предсказать


In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [26]:
songs = pd.read_json('song_info.json', lines=True) # откроем файл с информацей о песнях

In [29]:
# достанем все теги
tags = []

for el in songs["tags"]:
    for elem in el:
        if not(elem in tags):
            tags.append(elem)

In [40]:
data = songs[["tags", "views", "release_date", "pyongs"]] # отберем другие данные, которые нам нужны

In [44]:
# найдем, есть ли в конкретной песне каждый конкретный жанр
def search_tag(l, tag):
    if tag in l:
        return 1
    else:
        return 0

for t in tags:
    data[t] = songs["tags"].apply(search_tag, tag = t)

MemoryError: Unable to allocate 1.31 GiB for an array with shape (787, 223257) and data type int64

In [42]:
# приведем данные в нормальный вид, прологарифмируем целевую переменную - будем предсказывать порядок

def get_year(s):
    if s is not None:
        return int(s[-5:])
    else:
        return 1990
data["release_date"] = data["release_date"].apply(get_year)

data["views"] = np.log(data["views"].astype(int, errors = 'ignore'))
data["pyongs"] = data["pyongs"].astype(int, errors = 'ignore')

In [43]:
# задаем иксы и игреки
X = data[data.columns[2:]]
y = data["views"]
# делим выборку на трейн и тест
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3)

MemoryError: Unable to allocate 1.31 GiB for an array with shape (786, 223257) and data type int64

In [38]:
# инициализируем модель, передаем ей параметры
song_regression_model = LinearRegression()
song_regression_model.fit(X_tr, y_tr)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# пробуем что-то предсказать
preds_tr = song_regression_model.predict(X_tr)
preds_te = song_regression_model.predict(X_te)